In [1]:
import os
os.environ['CUPY_ACCELERATORS'] = 'cutensor'
import numpy as np
import cupy as cp
from scipy import fft
import zarr
from matplotlib import pyplot, colors, cm
from PIL import Image
from IPython.display import display
from tqdm.notebook import tqdm, trange

mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

def bytesize_string(nbytes):
    unit =          ['B',  'KiB' ,  'MiB' ,  'GiB' ,  'TiB' ,  'PiB' ,  'EiB' ,  'ZiB' ,  'YiB' ]
    size = np.array([ 1 , 1/2**10, 1/2**20, 1/2**30, 1/2**40, 1/2**50, 1/2**60, 1/2**70, 1/2**80]) * nbytes
    order_of_magnitude = np.argmax(size < 1.0) - 1
    return '{} {}'.format(size[order_of_magnitude], unit[order_of_magnitude])

def array_stats(a):
    print('{} × {} = {} | min: {}, max: {}, avg: {}'.format(a.shape, a.dtype, bytesize_string(a.nbytes), np.amin(a), np.amax(a), np.average(a)))

def display_image(values, color_map=None):
    values = values if color_map is None else cm.get_cmap(color_map)(values)
    display(Image.fromarray((np.flip(np.swapaxes(values, 0, 1), axis=0) * 255.0).astype(np.uint8)))
    
def save_image(name, values, color_map=None):
    values = values if color_map is None else cm.get_cmap(color_map)(values)
    Image.fromarray((np.flip(np.swapaxes(values, 0, 1), axis=0) * 255.0).astype(np.uint8)).save('figures/{}.png'.format(name))

In [2]:
# Focus range
n_f = 100
zeta_F = np.linspace(0.0, 5.0, num=n_f) / 1000

In [3]:
n_r = 1024

n_u = 20
n_v = 1080
chunk_v = 40
n_chunk = n_v//chunk_v

n_res = 4

n_a = np.arange(1, n_res+1, dtype=np.uint32) * 12
n_b = np.arange(1, n_res+1, dtype=np.uint32) * 140

# Light field groups.
chunk_group = 2
n_group = n_chunk - (chunk_group - 1)

In [4]:
continuous_projections = []
continuous_autocorrelations = []

for index_res in [0, 1, 2, 3]:
    projection = zarr.open('data/matrices/projection-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r')
    autocorrelation = zarr.open('data/matrices/autocorrelation-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r')

    n_ab = projection.shape[1]

    continuous_projections.append(np.zeros((n_ab, n_u, n_chunk, chunk_v), dtype=np.float32))
    continuous_autocorrelations.append(np.zeros((n_ab, n_ab), dtype=np.float32))

    for index_f in range(n_f):
        continuous_projections[-1] += projection[index_f] / n_f
        continuous_autocorrelations[-1] += autocorrelation[index_f] / n_f

In [5]:
def run_simulation(scene, index_res, n_iter, focus_index, known_focus):
    light_field = zarr.open('data/' + scene + '_sampled.zarr', mode='r')
    projection = zarr.open('data/matrices/projection-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r')
    autocorrelation = zarr.open('data/matrices/autocorrelation-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r')
    simulation = zarr.open('data/matrices/simulation-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r')

    n_ab = projection.shape[1]
        
    if known_focus:
        projection = projection[focus_index]
        autocorrelation = autocorrelation[focus_index]
    else:
        projection = continuous_projections[index_res]
        autocorrelation = continuous_autocorrelations[index_res]

    simulation = simulation[focus_index]

    # Assign each element to a group.
    chunk_incidence_per_element = np.any(projection, axis=(1, 3))
    element_assignment_per_group = np.minimum(np.argmax(chunk_incidence_per_element, axis=1), n_group-1).reshape(1, n_ab) == np.arange(n_group).reshape(n_group, 1)
    element_indices_per_group = [np.nonzero(element_assignment)[0] for element_assignment in element_assignment_per_group]

    ################################################################################################

    # Load the light field and the projection matrix into the GPU
    light_field_GPU = cp.array(light_field[:])
    projection_GPU = cp.array(projection[:])

    # Projection coefficients to be computed
    projection_coef_GPU = cp.zeros((n_ab, n_ab, 3), dtype=cp.float32)

    for idx_group in range(n_group):
        slice_x = slice(idx_group, idx_group + chunk_group)
        for idy_group in range(n_group):
            slice_y = slice(idy_group, idy_group + chunk_group)

            # Select sample values for this group.
            sample_values_GPU = light_field_GPU[:, slice_x, :, :, slice_y, :].reshape(n_u * chunk_group * chunk_v, n_u * chunk_group * chunk_v, 3)

            # Display elements assigned to this group.
            idx_element = element_indices_per_group[idx_group]
            idy_element = element_indices_per_group[idy_group]

            # Compute projection coefficients.
            lm_x_GPU = projection_GPU[idx_element][:, :, slice_x, :].reshape(-1, n_u * chunk_group * chunk_v)
            lm_y_GPU = projection_GPU[idy_element][:, :, slice_y, :].reshape(-1, n_u * chunk_group * chunk_v)
            projection_coef_GPU[np.ix_(idx_element, idy_element)] = cp.einsum('hx,vy,xyc->hvc', lm_x_GPU, lm_y_GPU, sample_values_GPU)

    # Free GPU memory
    del sample_values_GPU
    del lm_x_GPU
    del lm_y_GPU
    del light_field_GPU
    del projection_GPU
    mempool.free_all_blocks()

    ################################################################################################

    autocorrelation_GPU = cp.array(autocorrelation[:])

    rng = cp.random.default_rng()

    coefficients_GPU = 1.0 - rng.random((n_ab, n_ab, 3), dtype=cp.float32)
    # For each multiplicative rule iteration...
    for index_iter in range(n_iter):
        # Apply the autocorrelation linear map on current coefficients.
        temp = cp.tensordot(autocorrelation_GPU, cp.tensordot(autocorrelation_GPU, coefficients_GPU, axes=(1, 1)), axes=(1, 1))
        # Divide the projected coefficients by the autocorrelated coefficients.
        temp = cp.add(temp, cp.finfo(cp.float32).eps, out=temp)
        temp = cp.divide(projection_coef_GPU, temp, out=temp)
        # Update current coefficients with the multiplicative rule.
        coefficients_GPU = cp.multiply(coefficients_GPU, temp, out=coefficients_GPU)
        # Clip coefficients to interval [0, 1].
        coefficients_GPU = cp.clip(coefficients_GPU, 0.0, 1.0, out=coefficients_GPU)

    # Free GPU memory
    del autocorrelation_GPU
    del temp
    del projection_coef_GPU
    mempool.free_all_blocks()

    ################################################################################################

    simulation_GPU = cp.array(simulation[:])

    retinal_image = cp.tensordot(simulation_GPU, cp.tensordot(simulation_GPU, coefficients_GPU, axes=(1, 1)), axes=(1, 1)).get()

    # Free GPU memory
    del simulation_GPU
    del coefficients_GPU
    mempool.free_all_blocks()

    return retinal_image

In [6]:
with tqdm(total=3 * 8 * 24) as pbar:
    for index_res in [0, 1, 2]:
        for n_iter in [1, 5, 10, 15, 20, 30, 40, 50]:
            retinal_image = run_simulation('car', index_res, n_iter, 60, True)
            save_image('multiter/car-{}x{}-near-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('car', index_res, n_iter, 60, False)
            save_image('multiter/car-{}x{}-near-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('car', index_res, n_iter, 25, True)
            save_image('multiter/car-{}x{}-mid-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('car', index_res, n_iter, 25, False)
            save_image('multiter/car-{}x{}-mid-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('car', index_res, n_iter, 10, True)
            save_image('multiter/car-{}x{}-far-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('car', index_res, n_iter, 10, False)
            save_image('multiter/car-{}x{}-far-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()


            retinal_image = run_simulation('chess', index_res, n_iter, 85, True)
            save_image('multiter/chess-{}x{}-near-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('chess', index_res, n_iter, 85, False)
            save_image('multiter/chess-{}x{}-near-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('chess', index_res, n_iter, 65, True)
            save_image('multiter/chess-{}x{}-mid-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('chess', index_res, n_iter, 65, False)
            save_image('multiter/chess-{}x{}-mid-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('chess', index_res, n_iter, 30, True)
            save_image('multiter/chess-{}x{}-far-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('chess', index_res, n_iter, 30, False)
            save_image('multiter/chess-{}x{}-far-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()


            retinal_image = run_simulation('dragon', index_res, n_iter, 65, True)
            save_image('multiter/dragon-{}x{}-near-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('dragon', index_res, n_iter, 65, False)
            save_image('multiter/dragon-{}x{}-near-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('dragon', index_res, n_iter, 45, True)
            save_image('multiter/dragon-{}x{}-mid-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('dragon', index_res, n_iter, 45, False)
            save_image('multiter/dragon-{}x{}-mid-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('dragon', index_res, n_iter, 25, True)
            save_image('multiter/dragon-{}x{}-far-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('dragon', index_res, n_iter, 25, False)
            save_image('multiter/dragon-{}x{}-far-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()


            retinal_image = run_simulation('sponza', index_res, n_iter, 80, True)
            save_image('multiter/sponza-{}x{}-near-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('sponza', index_res, n_iter, 80, False)
            save_image('multiter/sponza-{}x{}-near-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('sponza', index_res, n_iter, 20, True)
            save_image('multiter/sponza-{}x{}-mid-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('sponza', index_res, n_iter, 20, False)
            save_image('multiter/sponza-{}x{}-mid-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

            retinal_image = run_simulation('sponza', index_res, n_iter, 0, True)
            save_image('multiter/sponza-{}x{}-far-known-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()
            retinal_image = run_simulation('sponza', index_res, n_iter, 0, False)
            save_image('multiter/sponza-{}x{}-far-unknown-i{}'.format(n_a[index_res], n_b[index_res], n_iter), retinal_image)
            pbar.update()

  0%|          | 0/576 [00:00<?, ?it/s]

KeyboardInterrupt: 